In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gower
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\ahtis\Documents\TUGraz\AdvancedIR\AIR\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the seed to get reproducible results

In [2]:
np.random.seed(123)

Data preparation. Reading the data in from file, merging the headers into one line, splitting the data into X and Y sets, and into test and train sets

In [3]:
headers = pd.read_csv("./data/tracks.csv", nrows = 3, header = None, sep=",")
combined_headers = headers.apply(lambda x: '.'.join(x.dropna().astype(str)), axis=0)
df = pd.read_csv("./data/tracks.csv", skiprows = 3, header = None, sep=",")
df.columns = combined_headers
print(df)

X = df.iloc[: ,1:]
Y = df.iloc[:, :1]

X_train,X_test,y_test,y_train = train_test_split(X,Y,test_size= 0.2)

        track_id  album.comments   album.date_created  album.date_released  \
0              2               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
1              3               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
2              5               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
3             10               0  2008-11-26 01:45:08  2008-02-06 00:00:00   
4             20               0  2008-11-26 01:45:05  2009-01-06 00:00:00   
...          ...             ...                  ...                  ...   
106569    155316               0  2017-03-30 15:20:35  2017-02-17 00:00:00   
106570    155317               0  2017-03-30 15:20:35  2017-02-17 00:00:00   
106571    155318               0  2017-03-30 15:20:35  2017-02-17 00:00:00   
106572    155319               0  2017-03-30 15:20:35  2017-02-17 00:00:00   
106573    155320               0  2017-03-26 16:22:18  2017-03-26 00:00:00   

       album.engineer  album.favorites  album.id  \
0          

In [4]:
# Retrieve query from the user
def get_user_query(df):
    print("Enter the attributes of the query as comma-separated values:")
    query_input = input()
    try:
        # Split input and convert to match DataFrame structure
        query_data = [float(x) if x.replace('.', '', 1).isdigit() else x for x in query_input.split(",")]
        query_df = pd.DataFrame([query_data], columns=df.columns)
        return query_df
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

Query retrieval, and running the Gower distance algorithm on the data 

In [5]:

query = get_user_query(X) 
if query is None:
    query = X.iloc[0:1]
    
distances = gower.gower_matrix(X, query)

print(distances)

Enter the attributes of the query as comma-separated values:
Error processing query: 52 columns passed, passed data had 3 columns
[[0.17307693]
 [0.23097612]
 [0.21187736]
 ...
 [       nan]
 [       nan]
 [       nan]]


Ranking rows in the DataFrame according to their proximity to the query. Excluding the 1st record from the closest_indices list, since it is an id of the record from the query


In [6]:
distances = np.nan_to_num(distances, nan=np.inf)
distances = distances.flatten()


closest_indices = np.argsort(distances)[1:10001]
print(closest_indices)

print(df.iloc[closest_indices])

[    2     1     9 ...  3428 36395  8343]
       track_id  album.comments   album.date_created  album.date_released  \
2             5               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
1             3               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
9           134               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
5729      10666               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
5846      10815               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
...         ...             ...                  ...                  ...   
7562      13173               0  2009-05-18 19:56:50  2002-01-01 00:00:00   
15533     24907               0  2010-02-08 16:39:46                  NaN   
3428       6522               1  2009-02-19 10:34:10  2008-10-28 00:00:00   
36395     54735               0  2011-10-12 23:43:16  2011-10-01 00:00:00   
8343      14179               0  2009-06-01 08:45:14                  NaN   

      album.engineer  album.favor

In [7]:
# Pre-trained light weight model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Combining textual columns for S-BERT
textual_data = (df['track_title'] + " " + df['track_information']).fillna("")
query_text = (query['track_title'].iloc[0] + " " + query['track_information'].iloc[0]).strip()

# Computing embeddings
text_embeddings = model.encode(textual_data.tolist())
query_embedding = model.encode([query_text])

c:\Users\ahtis\Documents\TUGraz\AdvancedIR\AIR\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahtis\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyError: 'track_title'

In [ ]:
# Filter dataset using closest_indices from Gower
filtered_embeddings = text_embeddings[closest_indices]

# Compute cosine similarity
similarity_scores = cosine_similarity(query_embedding, filtered_embeddings).flatten()

# Get the top N results
top_indices = np.argsort(similarity_scores)[::-1][:10]
final_indices = [closest_indices[i] for i in top_indices]

In [ ]:
# Retrieve recommendations
recommendations = df.iloc[final_indices]
print(recommendations[['track_title', 'track_information', 'artist_name']])